In [2]:
"""MEV Specification: CNL from base model with variable alphas.

Swiss Metro Case Study
Demand Modeling: 1.202
Spring 2019
Alexandra Berke

"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models

pandas = pd.read_table("../Data/swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',0,-20,20,0)
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1)
ASC_SM	 = Beta('ASC_SM',0,-20,20,0)
B_CAR_COST	 = Beta('B_CAR_COST',0,-10,10,0)
B_HE	 = Beta('B_HE',0,-10,10,0)
B_SM_COST	 = Beta('B_SM_COST',0,-10,10,0)
B_CAR_TIME	 = Beta('B_CAR_TIME',0,-10,10,0)
B_TRAIN_TIME	 = Beta('B_TRAIN_TIME',0,-10,10,0)
B_SM_TIME	 = Beta('B_SM_TIME',0,-10,10,0)
B_TRAIN_COST	 = Beta('B_TRAIN_COST',0,-10,10,0)
B_SENIOR	 = Beta('B_SENIOR',0,-10,10,0)
B_GA	 = Beta('B_GA',0,-10,80,0)
B_FIRST	 = Beta('B_FIRST',0,-10,80,0)

# scale parameters for the nests
MU_rail = Beta('MU_rail',1.5,1,10,0)
MU_existing = Beta('MU_existing',1.5,1,10,0)

# alpha parameters for the nests
alpha_rail_SM = Beta('alpha_rail_SM',1,0,1,1)
alpha_rail_train = Beta('alpha_rail_train',0.5,0,1,0)
alpha_existing_car = Beta('alpha_existing_car',1,0,1,1)
alpha_existing_train = 1 - alpha_rail_train

# Define here arithmetic expressions for name that are not directly 
# available from the data

SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

#Utilities
Car_SP = ASC_CAR * one + B_CAR_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR
SBB_SP = ASC_SBB * one + B_TRAIN_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
        + B_GA * GA + B_FIRST * FIRST
SM_SP = ASC_SM * one + B_SM_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
        + B_SENIOR * SENIOR + B_FIRST * FIRST

V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 6  ))>0
database.remove(exclude)


# definition of nests
alpha_rail = {1: alpha_rail_train, 2: alpha_rail_SM, 3: 0}
alpha_existing = {1: alpha_existing_train, 2: 0, 3: alpha_existing_car}

nest_rail = MU_rail, alpha_rail
nest_existing = MU_existing, alpha_existing

nests = nest_existing, nest_rail

# CNL (Cross-Nested Logit Model), with availability conditions
logprob = models.logcnl_avail(V,av,nests,CHOICE)

biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "MEV_SM_base_CNL_var"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_CAR=	-0.346
ASC_SM=	-0.11
B_CAR_COST=	-0.00581
B_CAR_TIME=	-0.0102
B_FIRST=	0.235
B_GA=	0.603
B_HE=	-0.00127
B_SENIOR=	-0.344
B_SM_COST=	-0.0119
B_SM_TIME=	-0.00322
B_TRAIN_COST=	-0.019
B_TRAIN_TIME=	-0.00226
MU_existing=	1.15
MU_rail=	7.12
alpha_rail_train=	0.892
                     Value   Std err     t-test       p-value  Rob. Std err  \
ASC_CAR          -0.346400  0.094598  -3.661809  2.504403e-04      0.106673   
ASC_SM           -0.110384  0.031768  -3.474644  5.115323e-04      0.041737   
B_CAR_COST       -0.005808  0.001026  -5.661208  1.503111e-08      0.001518   
B_CAR_TIME       -0.010217  0.000846 -12.072647  0.000000e+00      0.001504   
B_FIRST           0.235396  0.070222   3.352190  8.017499e-04      0.075432   
B_GA              0.602871  0.178669   3.374224  7.402400e-04      0.195852   
B_HE             -0.001274  0.000270  -4.711777  2.455663e-06      0.000366   
B_SENIOR         -0.344111  0.044905  -7.663063  1.820766e-14      0.072016   
B_SM_COST        -0.